In [161]:
import torch.nn.functional as F 
import pickle, gzip, math, torch, matplotlib as mpl
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'
import torch.nn as nn


from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from IPython import embed

In [162]:
mpl.rcParams['image.cmap'] = 'gray'

In [163]:
def get_data(): 
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _)= pickle.load(f,encoding='latin-1')
    return map(tensor,(x_train, y_train ,x_valid, y_valid))


def normalize(x, m, s): 
    return (x-m)/s


In [164]:
x_train, y_train ,x_valid, y_valid = get_data()

In [165]:
n,m = x_train.shape
c = y_train.max() + 1
nh = 50 

In [166]:
class Model(nn.Module):
    def __init__(self, n_in,nh,n_out): 
        super().__init__()
        self.layers = [nn.Linear(n_in,nh),nn.ReLU(),nn.Linear(nh,n_out)]
        
    def __call__(self,x):
        for l in self.layers: 
            x=l(x)
        return x

In [167]:
model = Model(m,nh,10)

In [168]:
pred = model(x_train)

# Cross entropy loss

\begin{equation*}
\sigma(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}} 
\end{equation*}

Th soft max is as a above, however before we have to remember that pytorch likes softmax log. 


In [169]:
pred.shape

torch.Size([50000, 10])

In [170]:
def log_softmax(x): 
    return (x.exp()/(x.exp().sum(-1, keepdim=True))).log()
    


The fucntion is executed for every class in every row seperate in then both of the dicison is the exp sum of all classes for that row

In [171]:
sm_pred = log_softmax(pred)



losses for all diffrent categories above! 

In [172]:
y_train[:3]


tensor([5, 0, 4])

In [173]:
sm_pred

tensor([[-2.3882, -2.2585, -2.1809,  ..., -2.4748, -2.1314, -2.4649],
        [-2.4065, -2.2363, -2.2153,  ..., -2.4807, -2.1426, -2.4401],
        [-2.4666, -2.2771, -2.1021,  ..., -2.4079, -2.1172, -2.5161],
        ...,
        [-2.4355, -2.2326, -2.1154,  ..., -2.4659, -2.0998, -2.4899],
        [-2.4380, -2.2830, -2.1768,  ..., -2.4240, -2.1793, -2.4328],
        [-2.4642, -2.3472, -2.0794,  ..., -2.3011, -2.1986, -2.5288]],
       grad_fn=<LogBackward>)

In [174]:
sm_pred[[0,1,2], [5,0,4]]


tensor([-2.2423, -2.4065, -2.3328], grad_fn=<IndexBackward>)

In [175]:
def nll(input, target): 
    return -input[range(target.shape[0]), target].mean()

In [176]:
range(y_train.shape[0])

range(0, 50000)

In [177]:
-sm_pred[range(y_train.shape[0]), y_train]

tensor([2.2423, 2.4065, 2.3328,  ..., 2.0998, 2.2878, 2.1986],
       grad_fn=<NegBackward>)

In [178]:
def nll(input, target): 
    return -input[range(target.shape[0]), target].mean()

In [179]:
loss = nll(sm_pred, y_train)


In [180]:
loss

tensor(2.3131, grad_fn=<NegBackward>)

Simplification of the log soft max 

In [181]:
def log_softmax(x): 
    return x - x.exp().sum(-1,keepdim=True).log()


more stable way of calculating! 

In [182]:
def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"
def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)

def test_near(a,b): 
    test(a,b,near)



In [183]:
test_near(nll(log_softmax(pred), y_train), loss)
loss

tensor(2.3131, grad_fn=<NegBackward>)

In [184]:
nll(log_softmax(pred), y_train)

tensor(2.3131, grad_fn=<NegBackward>)

In [185]:
F.nll_loss(F.log_softmax(pred,-1),y_train)

tensor(2.3131, grad_fn=<NllLossBackward>)

In [186]:
F.cross_entropy(pred,y_train)

tensor(2.3131, grad_fn=<NllLossBackward>)

#### Cross entropy loss 

The coss entropy loss is calculating the softmax, log and then taking the max. From the max class, class with highets provavility we calculate. That is the cross entropy loss

## Basic training loop

1. Get the output of the model on a batch of inputs 
2. Compare the output to the y_train
3. Calculate the gradient of the loss with respect to every parameter
4. Update the said parameters eith those graidents to make them a little bit better

In [187]:
loss_func = F.cross_entropy

In [188]:

def accuracy(out,yb):
    return (torch.argmax(out,dim=1)==yb).float().mean()

In [189]:
bs=64

xb = x_train[0:bs]
preds = model(xb)
preds[0],preds.shape

(tensor([-0.0427,  0.0870,  0.1646,  0.0352, -0.0202,  0.1032,  0.0777, -0.1293,
          0.2141, -0.1194], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [190]:
yb = y_train[0:bs]

In [191]:
loss_func(preds,yb)

tensor(2.3354, grad_fn=<NllLossBackward>)

In [192]:
accuracy(preds,y_train[0:bs])

tensor(0.0312)

In [193]:
lr = 0.5
epochs = 1

In [194]:
for epoch in range(epochs): 
    for i in range(((n-1)//bs+1)):
        start_i=i*bs
        end_i = start_i + bs 
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss = loss_func(model(xb), yb)
        loss.backward()
        with torch.no_grad():
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias   -= l.bias.grad   * lr
                    l.weight.grad.zero_()
                    l.bias  .grad.zero_()    

In [195]:
loss_func(model(xb), yb), accuracy(model(xb), yb)


(tensor(0.0666, grad_fn=<NllLossBackward>), tensor(1.))

# Using parameter and optim 

In [196]:
class Model(nn.Module):
    def __init__ (self,n_in,nh,n_out):
        super().__init__()
        # The super init imports all the sub stuff from nn.Module
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)
        
    def __call__(self,x):
        # The call function means that we can call the class as a function with the name
        return self.l2(F.relu(self.l1(x)))

In [197]:
model = Model(m, nh, 10)


In [198]:
for name,l in model.named_children(): 
    print(f"{name}:{l}")

l1:Linear(in_features=784, out_features=50, bias=True)
l2:Linear(in_features=50, out_features=10, bias=True)


In [199]:
model

Model(
  (l1): Linear(in_features=784, out_features=50, bias=True)
  (l2): Linear(in_features=50, out_features=10, bias=True)
)

In [200]:
model.l1

Linear(in_features=784, out_features=50, bias=True)

In [201]:
def fit(): 
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb)
            loss.backward()
            # torch.no_grad set the all gradients in block to false
            # Will remove the memory so we can do back proipargation on the stuff we do after! 
            with torch.no_grad(): 
                for p in model.parameters(): 
                    p -= p.grad * lr 
                    model.zero_grad()
                
            

In [202]:
fit() 
loss_func(model(xb),yb),accuracy(model(xb),yb)

(tensor(0.4047, grad_fn=<NllLossBackward>), tensor(0.8750))

Behind the scenes, PyTorch overrides the __setattr__ function in nn.Module so that the submodules you define are properly registered as parameters of the model.



In [207]:
class DummyModule(): 
    def __init__(self,n_in,nh,n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)
    
    # This sets the stuff about the naming
    def __setattr__(self,k,v): 
        if not k.startswith("_"): 
            self._modules[k] = v
        super().__setattr__(k,v)      

    def __repr__(self): 
        return f'{self._modules}'
    
    def parameters(self):
        for l in self._modules.values(): 
            for p in l.parameters(): 
                yield p

In [208]:
mdl = DummyModule(m,nh,10)
mdl

{'l1': Linear(in_features=784, out_features=50, bias=True), 'l2': Linear(in_features=50, out_features=10, bias=True)}

In [210]:
# The rotation here is oposit to how it normaly is
[o.shape for o in mdl.parameters()]

[torch.Size([50, 784]),
 torch.Size([50]),
 torch.Size([10, 50]),
 torch.Size([10])]

# Registering modules

In [214]:
layers = [nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10)]

In [215]:
class Model(nn.Module): 
    def __init__(self,layers): 
        super().__init__()
        self.layers = layers
        for i,l in enumerate(self.layers): 
            self.add_module(f'layer_{i}', l)
    
    def __call__(self,x): 
        for l in self.layers: 
            x=l(x)
        return x 

In [216]:
model = Model(layers)

In [217]:
model

Model(
  (layer_0): Linear(in_features=784, out_features=50, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=50, out_features=10, bias=True)
)

In [218]:
class SequentialModel(nn.Module):
    def __init__(self,layers): 
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def __call__(self,x): 
        for l in self.layers: 
            x = l(x)
        return x 

In [219]:
model = SequentialModel(layers)

In [220]:
model

SequentialModel(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [221]:
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.4592, grad_fn=<NllLossBackward>), tensor(0.8750))

# nn.Sequential

In [225]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [226]:
fit()
loss_func(model(xb), yb), accuracy(model(xb), yb)


(tensor(0.3632, grad_fn=<NllLossBackward>), tensor(0.8750))

In [228]:
model

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)

# Writting the optim

In [229]:
class Optimizer(): 
    
    def __init__(self,params,lr=0.5): 
        self.params = list(params) 
        self.lr = lr 
    
    def step(self): 
        with torch.no_grad(): 
            for p in self.params: 
                p -= p.grad * lr
    
    def zero_grad(self): 
        for p in self.params:
            p.grad.data.zero_()

In [230]:
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [231]:
opt = Optimizer(model.parameters())

In [232]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()
        

In [233]:
loss,acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
loss,acc

(tensor(0.0615, grad_fn=<NllLossBackward>), tensor(1.))

In [234]:
from torch import optim


In [235]:
def get_model():
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))
    return model, optim.SGD(model.parameters(), lr=lr)

In [236]:
model,opt = get_model()
loss_func(model(xb), yb)

tensor(2.3219, grad_fn=<NllLossBackward>)

In [239]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

In [240]:
loss,acc = loss_func(model(xb), yb), accuracy(model(xb), yb)
loss,acc

(tensor(0.0131, grad_fn=<NllLossBackward>), tensor(1.))

In [241]:
assert acc>0.7